# Jsonformer in 20 lines of code

This notebook shows how language specific formatting methods can be implemented using `guidance`. We use <a href="https://github.com/1rgs/jsonformer">Jsonformer</a> as an example, where below we write a short mapping function that takes in a Jsonformer-style input and returns an executable Guidance program. This program then benefits from built-in `guidance` functionality like guidance acceleration and token healing.

Similar methods can be used to generate output in other formats, like XML, YAML, etc. Note that this is not a deeply tested peice of code, just an example of the flexibility you have to use `guidance` as the backend for your own language-specific formatting methods.

## Schema transformation code

In [1]:
# define a recursive function that takes a jsonformer schema and returns a guidance program
def jsonformer2guidance(schema, key=None, indent=0):
    out = ""
    if schema['type'] == 'object':
        out += "  "*indent + "{\n"
        for k,v in schema['properties'].items():
            out += "  "*(indent+1) + k + ": " + jsonformer2guidance(v, k, indent+1) + ",\n"
        out += "  "*indent + "}"
        return out
    elif schema['type'] == 'array':
        if 'max_items' in schema:
            extra_args = f" max_iterations={schema['max_items']}"
        else:
            extra_args = ""
        return "[{{#geneach '"+key+"' stop=']'"+extra_args+"}}{{#unless @first}}, {{/unless}}" + jsonformer2guidance(schema['items'], "this") + "{{/geneach}}]"
    elif schema['type'] == 'string':
        return "\"{{gen '"+key+"' stop='\"'}}\""
    elif schema['type'] == 'number':
        return "{{gen '"+key+"' pattern='[0-9\\.]' stop=','}}"
    elif schema['type'] == 'boolean':
        return "{{#select '"+key+"'}}True{{or}}False{{/select}}"

## Example usage

In [2]:
import guidance

# we want to use an open model so we can get guidance acceleration and token healing
guidance.llm = guidance.llms.transformers.MPTChat('mosaicml/mpt-7b', device=1)

# define a jsonformer schema
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

# run the guidance program
prompt = "Generate a person's information based on the following schema:"
program = guidance(prompt + "\n" + jsonformer2guidance(json_schema))
out = program()

Stop program Generate a person's information based on the following schema:
{
 name: " John Doe ",
 age: 5 ,
 is_student: True ,
 courses: [ " Math " , " English " , " History " ],
}

In [4]:
# we can access all the generated variables through the executed program object
out.variables()

{'llm': <guidance.llms.transformers._mpt.MPTChat at 0x7f4ad86c71c0>,
 'name': 'John Doe',
 'age': '5',
 'is_student': 'True',
 'courses': ['Math', 'English', 'History']}

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>